# Config Maps

There are several ways to manage environment variables in Kubernetes. One of the most common ways is to use ConfigMaps. ConfigMaps allow us to decouple our configurations from our container images, which is important because we don't want to have to rebuild our images every time we want to change a configuration value.

In a Dockerfile we can set environment variables like this:

```python
ENV PORT=3000
```

The trouble is, that means that everyone using that image will have to use port 3000. It also means that if we want to change the port, we have to rebuild the image.

Let's create one first, call it `api-configmap.yaml`. Add the following YAML:

- `apiVersion: v1`
- `kind: ConfigMap`
- `metadata/name: synergychat-api-configmap`
- `data/API_PORT: "8080"`

Remember to apply it and check all the available configmap to verify:

```
kubectl get configmaps
```

Note that we haven't yet connected the config map to our pod, so it should still be crashing

# Applying the Config Map

Now, re-open the `api-deployment.yaml` file. We're going to add a few things to it. Under the `containers` section, add the following to the first (and only) entry:

```python
env:
  - name: API_PORT
    valueFrom:
      configMapKeyRef:
        name: synergychat-api-configmap
        key: API_PORT
```

This tells Kubernetes to set the API_PORT environment variable to the value of the API_PORT key in the synergychat-api-configmap config map. Reference the [official docs](https://kubernetes.io/docs/concepts/configuration/configmap/) if you're confused about the structure of the yaml.

Once it's applied, you should be able to take a look at the pods and see that a new API pod has been deployed and isn't crashing! (APLLY IT FIRST)

Then we forward the pod's API to `8080` in our local machine:

```python
kubectl port-forward <pod-name> 8080:8080
```



Run http://localhost:8080 in the browser to see if it correct or not

# Config Maps Are Insecure

ConfigMaps are a great way to manage innocent environment variables in Kubernetes. Things like:
- Ports
- URLs of other services
- Feature flags
- Settings that change between environments, like `DEBUG` mode

However, they are *not cryptographically secure*. ConfigMaps aren't encrypted, and they can be accessed by anyone with access to the cluster.

If you need to store sensitive information, you should use [Kubernetes Secrets](https://kubernetes.io/docs/concepts/configuration/secret/) or a third-party solution.

# Crawler

We've got one last application to deploy to our cluster: the crawler. This is an application that continuously crawls [Project Gutenberg](https://www.gutenberg.org/) and exposes the juicy data that it finds via a JSON API.

Create a copy of your `api-configmap.yaml` file and call it `crawler-configmap.yaml`. We're going to make a few changes to it:
1. Replace the `name` to synergychat-crawler-configmap
2. Remove the `API_PORT` variable
3. Add:
    - `CRAWLER_PORT: "8080"`
    - `CRAWLER_KEYWORDS: "love,hate,joy,sadness,anger,disgust,fear,surprise"`

It's okay that the `CRAWLER_PORT` in the crawler deployment is the same as the `API_PORT` in the api deployment. They're in different pods, and these are pod-internal ports.

Same as above, create a copy of your `api-deployment.yaml` file and call it `crawler-deployment.yaml` then:
1. Update all `synergychat-api` references to `synergychat-crawler`.
2. Update the image URL to `bootdotdev/synergychat-crawler:latest`.
3. Update the environment variable references to match the new config map. (See below)

We can use the same format of the `api-deployment.yaml`:

```yaml
spec:
  containers:
    - image: bootdotdev/synergychat-crawler:latest
      name: synergychat-crawler
      env:
        - name: CLAWLER_PORT
          valueFrom:
            configMapKeyRef:
              name: synergychat-crawler-configmap
              key: CLAWLER_PORT
```

But to preventt this "Hard" format and keep it clean, we should list it out like:

```yaml
spec:
  containers:
    - image: image_url
      name: name_space
      env:
        - name: THING_ONE
          valueFrom:
            configMapKeyRef:
              name: <name>
              key: <key>
        - name: THING_TWO
          valueFrom:
            configMapKeyRef:
              name: <name>
              key: <key>
        - name: THING_THREE
          ...
```

Beside we can use the `envFrom` key instead of the `env` key to reference the entire config map and make it available to the pods in the deployment, in our case it will be:

```yaml
spec:
  containers:
    - name: synergychat-clawler
      image: bootdotdev/synergychat-crawler:latest
      envFrom:
        - configMapRef:
            name: synergychat-crawler-configmap
```

This injects all keys from the `synergychat-crawler-configmap` into the container as environment variables. Update then run it